In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('yandex_music_project.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


In [3]:
df = df.rename(columns={
    '  userID': 'user_id',
    'Track': 'track',
    '  City  ': 'city',
    'Day': 'day'})
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

In [4]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

In [5]:
df = df.fillna('unknown')
genre_df = df.dropna(subset=['genre'])
genre_df = genre_df.fillna('unknown')
print(df.isna().sum(), genre_df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64 user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


In [6]:
df.duplicated().sum()

3826

In [7]:
genre_df.duplicated().sum()

3826

In [8]:
df = df.drop_duplicates().reset_index(drop=True)
genre_df = genre_df.drop_duplicates().reset_index(drop=True)
print(df.duplicated().sum(), genre_df.duplicated().sum())

0 0


In [9]:
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'alternativepunk',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'author',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'chanson',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',


In [10]:
df = df.replace({"türk": "türkçe",
                "soul": "soulful",
                "new": "newage",
                "latin": "latino",
                "hip-hop": "hiphop",
                 "hip": "hiphop",
                "frankreich": "französisch",
                "africa": "afrikaans",
                "электроника": "electronic"})
genre_df = genre_df.replace({"türk": "türkçe",
                "soul": "soulful",
                "new": "newage",
                "latin": "latino",
                "hip-hop": "hiphop",
                "frankreich": "französisch",
                "africa": "afrikaans",
                "электроника": "electronic"})

Закончили предобработку данных:
1. Переименовали столбцы, чтобы с ними было удобней работать
2. Удалили все явные дубликаты
3. Убрали все неявные дубликаты

Проверим первую гипотезу.
Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
Для разных гипотез я сделал 2 разных DataFrame. Для первой гипотезы нам не важны жанры, а только активность, поэтому нам нужно больше данных. 
Для двух других гипотез, я удалил строки с неизвестными жанрами.

In [11]:
df.groupby('day')['user_id'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

In [22]:
moscow_activity = df[df['city'] == 'Moscow']
md = moscow_activity.groupby('day')['user_id'].count()
md

day
Friday       15945
Monday       15740
Wednesday    11056
Name: user_id, dtype: int64

In [26]:
spb_activity = df[df['city'] == 'Saint-Petersburg']
sd = spb_activity.groupby('day')['user_id'].count()
sd

day
Friday       5895
Monday       5614
Wednesday    7003
Name: user_id, dtype: int64

In [28]:
data = [['Moscow', md[1], md[2], md[0]],
        ['Saint-Petersburg', sd[1], sd[2], sd[0]]]
compresion = pd.DataFrame(columns=['city', 'Monday', 'Wednesday', 'Friday'], 
                          data = data)
display(compresion)

,city,Monday,Wednesday,Friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


Как видно из этой таблицы, в Москве музыку слушали гораздо больше, чем в Санкт-Петербурге. Так что гипотеза подтвердилась. Тут еще можно заметить интересную особенность. В Санкт-Петербурге в среду виден пик слушателей, тогда как в Москве мы наблюдаем два пика в Понедельник и Пятницу. Чтобы как-то лучше проанализировать это нам нужны данные на всю неделю.

Проверим вторую гипотезу.
Утром в понедельник в Москве преобладают одни жанры музыки, а в Петербурге — другие. Это верно и для вечера пятницы.

За утро мы будем считать промежуток с 8:00 до 11:00.
За вечер мы будем считать промежуток с 17:00 до 20:00. 

In [30]:
moscow_genres = genre_df[genre_df['city'] == 'Moscow']
spb_genres = genre_df[genre_df['city'] == 'Saint-Petersburg']

In [41]:
moscow_morning_monday = moscow_genres[moscow_genres['day'] == 'Monday']
moscow_morning_monday = moscow_morning_monday[moscow_morning_monday['time'] >= "08:00"]
moscow_morning_monday.groupby('genre')['genre'].count().sort_values(ascending=False)

genre
pop           2154
dance         1669
rock          1452
electronic    1432
hip            770
              ... 
horror           1
hop              1
historisch       1
gypsy            1
worldbeat        1
Name: genre, Length: 214, dtype: int64